In [ ]:
import io
import tarfile
import os
from dataset_api import Hot3dDataProvider
from data_loaders.loader_object_library import load_object_library
from data_loaders.mano_layer import MANOHandModel

from projectaria_tools.core.stream_id import StreamId
from data_loaders.headsets import Headset
from projectaria_tools.core.calibration import FISHEYE624
from tqdm import tqdm
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import cv2
import json

stream_id = StreamId("214-1")

def get_gt(time_stamp, seq_name):
    home = os.path.expanduser("~")
    hot3d_dataset_path = home + "/Dataset/ljh/dataset/hot3d/full-hot3d"

    sequence_path = os.path.join(hot3d_dataset_path, seq_name)
    
    object_library_path = home +"/Dataset/ljh/dataset/hot3d/assets"
    mano_hand_model_path = home + "/dir/mano_v1_2/models"

    if not os.path.exists(sequence_path) or not os.path.exists(object_library_path):
        print("Invalid input sequence or library path.")
        print("Please do update the path to VALID values for your system.")
        raise
    object_library = load_object_library(object_library_folderpath=object_library_path)

    mano_hand_model = None
    if mano_hand_model_path is not None:
        mano_hand_model = MANOHandModel(mano_hand_model_path)

    hot3d_data_provider = Hot3dDataProvider(
        sequence_folder=sequence_path,
        object_library=object_library,
        mano_hand_model=mano_hand_model,
    )
    
    return hot3d_data_provider
    
root = "/home/jeongho/Dataset/ljh/dataset/hot3d/clip_dataset/train_aria"
for tar_name in tqdm(os.listdir(root)):
    root_path = os.path.join(root, tar_name)

    # 초기 설정
    video_path = os.path.join("/home/jeongho/Dataset/ljh/video/clip-hot3d", tar_name.split(".")[0] + ".mp4")
    frame_size = None
    fps = 30
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')


    with tarfile.open(root_path, "r:*") as tar:
        tar_list = tar.getnames()

        time_stamp = (len(tar_list) - 1) // 8
        for idx, time in enumerate(range(time_stamp)):
            time = str(time).zfill(6)
            
            # 예시: 첫 번째 JSON 파일 읽기
            json_member = tar.getmember(f"{time}.info.json")
            json_content = tar.extractfile(json_member).read()
            data = json.loads(json_content)

            # 예시: 첫 번째 JPG 이미지 열기
            jpg_member = tar.getmember(f"{time}.image_214-1.jpg")
            img_data = tar.extractfile(jpg_member).read()
            image = Image.open(io.BytesIO(img_data))
            img = np.array(image)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            timestamp_ns, seq_name = data['ref_timestamp_ns'], data['sequence_id']
            # Retrieve the image data for a given timestamp
            
            if not idx:
                hot3d_data_provider = get_gt(timestamp_ns, seq_name)
                device_data_provider = hot3d_data_provider.device_data_provider
                frame_size = (img.shape[1], img.shape[0])  # (width, height)
                out = cv2.VideoWriter(video_path, fourcc, fps, frame_size)
            
            aria_eye_gaze_data = (
                device_data_provider.get_eye_gaze(timestamp_ns)
                if hot3d_data_provider.get_device_type() is Headset.Aria
                else None
            )
            
            if aria_eye_gaze_data is not None:

                # Reproject EyeGaze for raw images
                camera_model = FISHEYE624
                
                eye_gaze_reprojection_data = (
                    device_data_provider.get_eye_gaze_in_camera(
                        stream_id, timestamp_ns, camera_model=camera_model
                    )
                )
                
                if (
                    eye_gaze_reprojection_data is None
                    or not eye_gaze_reprojection_data.any()
                ):
                    continue
                
            # Retrieve the image data for a given timestamp

            cv2.circle(img = img, center = (int(eye_gaze_reprojection_data[0]), int(eye_gaze_reprojection_data[1])), radius= 10, color = (255, 255, 0), thickness=-1)
            out.write(img)
        out.release()
            
        


In [ ]:
#
# Section 4: Eye Gaze data (only for Aria data)
#
# Take home message
# - Eye Gaze data is only available for Aria sequences
# - Eye Gaze data is retrieved via the device_data_provider
# - Eye Gaze data is a 3D ray that can be reprojected at any desired depth in a given image
#
from tqdm import tqdm
from projectaria_tools.core.stream_id import StreamId
from data_loaders.headsets import Headset
from projectaria_tools.core.calibration import FISHEYE624
from projectaria_tools.core.sensor_data import TimeDomain, TimeQueryOptions


device_data_provider = hot3d_data_provider.device_data_provider
device_pose_provider = hot3d_data_provider.device_pose_data_provider
stream_id = StreamId("214-1")
timestamps = device_data_provider.get_sequence_timestamps()
object_pose_provider = hot3d_data_provider.object_pose_data_provider



# Init a rerun context
rr.init("Eye Gaze reprojection in RGB image")
rec = rr.memory_recording()

# Limit to the some timestamps
for timestamp_ns in tqdm(timestamps[:1000]):

   # 1. OBJECTS
    pose_with_dt = object_pose_provider.get_pose_at_timestamp(
        timestamp_ns=timestamp_ns,
        time_query_options=TimeQueryOptions.CLOSEST,
        time_domain=TimeDomain.TIME_CODE
    )
    object_pose_collection = pose_with_dt.pose3d_collection

    for uid in object_pose_collection.object_uid_list:
        pose = object_pose_collection.poses[uid].T_world_object
        mesh = object_library.get(uid)  # uid 기준으로 mesh 가져옴
        if mesh is None:
            continue

        rr.log(
            f"mesh/object/{uid}",
            rr.Mesh3D(
                vertex_positions=mesh.vertices,
                indices=mesh.indices,
                colors=mesh.vertex_colors,
                transform=pose,
            )
        )

    # 2. HANDS (좌우 각각)
    for hand_side in ["left", "right"]:
        mano_pose3d = hot3d_data_provider.get_mano_hand_pose(timestamp_ns, hand_side)
        if mano_pose3d is None:
            continue
        T_world_hand = mano_pose3d.T_world_hand
        verts, faces = mano_hand_model.get_hand_mesh(hand_side, mano_pose3d.pose_params, mano_pose3d.shape_params)

        rr.log(
            f"mesh/hand/{hand_side}",
            rr.Mesh3D(
                vertex_positions=verts,
                indices=faces,
                colors=[(255, 255, 255)],  # 흰 손
                transform=T_world_hand,
            )
        )

    # 3. GAZE VECTOR
    camera_model = FISHEYE624
    
    eye_gaze_reprojection_data, gaze_center_in_camera = (
        device_data_provider.get_eye_gaze_in_camera(
            stream_id, timestamp_ns, camera_model=camera_model
        )
    )
    
    T_world_camera = T_world_device @ extrinsics
    gaze_center_in_world = T_world_camera @ gaze_center_in_camera

    rr.log(
        "gaze/line",
        rr.Points3D(
            positions=[gaze_center_in_world],
            radii=0.04,
            colors=[(0, 128, 255)],
            labels=[uid],
        )
    )
rr.notebook_show()

  0%|          | 0/1000 [00:00<?, ?it/s]


AttributeError: 'ObjectLibrary' object has no attribute 'get'